# Thereshold based segmentation

In [1]:
# Simple 5x5 "image"
# 0 = black, 255 = white
image = [
    [0,   25,   0,   0,   0],
    [0, 255, 255, 255,  0],
    [0, 255, 255, 255,  0],
    [0, 255, 255, 255,  0],
    [0,   0,   0,   0,   0]
]


In [2]:
def threshold_image(img, threshold):
    height = len(img)
    width = len(img[0])

    # Create empty output image
    output = [[0 for _ in range(width)] for _ in range(height)]

    for i in range(height):
        for j in range(width):
            if img[i][j] >= threshold:
                output[i][j] = 255   # White
            else:
                output[i][j] = 0     # Black

    return output

# Apply threshold
threshold_value = 50
segmented = threshold_image(image, threshold_value)

# Print result
for row in segmented:
    print(row)


[0, 0, 0, 0, 0]
[0, 255, 255, 255, 0]
[0, 255, 255, 255, 0]
[0, 255, 255, 255, 0]
[0, 0, 0, 0, 0]


# Adaptive thresholding

Gray = 0.299*R + 0.587*G + 0.114*B


In [3]:
from PIL import Image

# ---------- Load real image ----------
img = Image.open("images\geo-image.jpg").convert("L")  # Convert to grayscale
width, height = img.size
pixels = img.load()

# ---------- Parameters ----------
window_size = 15   # neighborhood size (must be odd)
C = 5              # constant to fine-tune sensitivity

half = window_size // 2

# Create output image
output = Image.new("L", (width, height))
out_pixels = output.load()

# ---------- Adaptive Thresholding ----------
for i in range(width):
    for j in range(height):

        # Compute local mean in window around (i, j)
        sum_pixels = 0
        count = 0

        for x in range(i - half, i + half + 1):
            for y in range(j - half, j + half + 1):

                if 0 <= x < width and 0 <= y < height:
                    sum_pixels += pixels[x, y]
                    count += 1

        local_mean = sum_pixels / count

        # Apply adaptive threshold
        if pixels[i, j] > (local_mean - C):
            out_pixels[i, j] = 255
        else:
            out_pixels[i, j] = 0

# Save result
output.save("adaptive_threshold_output.png")
print("Done! Check adaptive_threshold_output.png")


<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\SATYAKI MANDAL\AppData\Local\Temp\ipykernel_2632\1089166394.py:4: SyntaxWarning: invalid escape sequence '\g'
  img = Image.open("images\geo-image.jpg").convert("L")  # Convert to grayscale
C:\Users\SATYAKI MANDAL\AppData\Local\Temp\ipykernel_2632\1089166394.py:4: SyntaxWarning: invalid escape sequence '\g'
  img = Image.open("images\geo-image.jpg").convert("L")  # Convert to grayscale


KeyboardInterrupt: 

# Ostu's thresolding

In [ ]:
from PIL import Image

# Load image in grayscale
img = Image.open("images\butterfly.jpg").convert("L")
width, height = img.size
pixels = img.load()

# Step 1: Compute histogram (256 bins)
hist = [0] * 256

for i in range(width):
    for j in range(height):
        hist[pixels[i, j]] += 1

# Step 2: Total pixels
total_pixels = width * height

# Step 3: Otsu's method
best_threshold = 0
max_variance = 0

for T in range(256):

    # Class 1 (0 to T)
    w0 = sum(hist[:T+1]) / total_pixels
    if w0 == 0:
        continue

    mean0 = sum(i * hist[i] for i in range(T+1)) / sum(hist[:T+1])

    # Class 2 (T+1 to 255)
    w1 = sum(hist[T+1:]) / total_pixels
    if w1 == 0:
        continue

    mean1 = sum(i * hist[i] for i in range(T+1, 256)) / sum(hist[T+1:])

    # Between-class variance
    variance = w0 * w1 * (mean0 - mean1) ** 2

    if variance > max_variance:
        max_variance = variance
        best_threshold = T

print("Optimal Threshold:", best_threshold)

# Step 4: Apply threshold
output = Image.new("L", (width, height))
out_pixels = output.load()

for i in range(width):
    for j in range(height):
        if pixels[i, j] > best_threshold:
            out_pixels[i, j] = 255
        else:
            out_pixels[i, j] = 0

output.save("otsu_output.png")
print("Saved otsu_output.png")


# Edge based Segmentation

# 1. Threshold-Based Segmentation - Global Thresholding

In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Global Thresholding
threshold = 128
segmented = (img >= threshold).astype(np.uint8) * 255

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Global Thresholding')
plt.show()

ModuleNotFoundError: No module named 'numpy'

# Adaptive Thresholding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Adaptive Thresholding (mean of neighborhood)
def adaptive_threshold(img, block_size=11, C=2):
    height, width = img.shape
    segmented = np.zeros_like(img)
    pad = block_size // 2
    padded = np.pad(img, pad, mode='edge')
    for i in range(height):
        for j in range(width):
            block = padded[i:i+block_size, j:j+block_size]
            threshold = np.mean(block) - C
            segmented[i, j] = 255 if img[i, j] > threshold else 0
    return segmented

segmented = adaptive_threshold(img)

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Adaptive Thresholding')
plt.show()

# Otsu’s Thresholding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Otsu's Thresholding
def otsu_threshold(img):
    hist = np.histogram(img.ravel(), bins=256, range=(0, 256))[0]
    total_pixels = img.size
    current_max = 0
    threshold = 0
    sum_total = np.sum(np.arange(256) * hist)
    sumB = 0
    wB = 0
    wF = 0
    for i in range(256):
        wB += hist[i]
        if wB == 0:
            continue
        wF = total_pixels - wB
        if wF == 0:
            break
        sumB += i * hist[i]
        mB = sumB / wB
        mF = (sum_total - sumB) / wF
        between_var = wB * wF * (mB - mF) ** 2
        if between_var > current_max:
            current_max = between_var
            threshold = i
    return threshold

threshold = otsu_threshold(img)
segmented = (img >= threshold).astype(np.uint8) * 255

# Display output
plt.imshow(segmented, cmap='gray')
plt.title(f'Otsu’s Thresholding (threshold={threshold})')
plt.show()

# Multi-level Thresholding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Multi-level Thresholding (3 levels)
segmented = np.zeros_like(img)
segmented[img < 85] = 0
segmented[(img >= 85) & (img < 170)] = 127
segmented[img >= 170] = 255

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Multi-level Thresholding')
plt.show()

# Histogram-based Thresholding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Histogram-based Thresholding (find valley between peaks)
hist = np.histogram(img.ravel(), bins=256, range=(0, 256))[0]
# Simple: find the minimum between the two highest peaks
peaks = np.argsort(hist)[-2:]  # indices of two highest
peaks.sort()
threshold = np.argmin(hist[peaks[0]:peaks[1]]) + peaks[0]

segmented = (img >= threshold).astype(np.uint8) * 255

# Display output
plt.imshow(segmented, cmap='gray')
plt.title(f'Histogram-based Thresholding (threshold={threshold})')
plt.show()

# 2. Edge-Based Segmentation - Roberts Edge Detector

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Roberts Edge Detector
height, width = img.shape
edges = np.zeros_like(img)
for i in range(height-1):
    for j in range(width-1):
        gx = img[i, j] - img[i+1, j+1]
        gy = img[i+1, j] - img[i, j+1]
        edges[i, j] = np.sqrt(gx**2 + gy**2)

# Threshold to binary
edges = (edges > np.mean(edges)).astype(np.uint8) * 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Roberts Edge Detector')
plt.show()

# Prewitt Edge Detector

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Prewitt Edge Detector
def prewitt_edge(img):
    height, width = img.shape
    edges = np.zeros_like(img)
    Gx = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    Gy = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            gx = np.sum(region * Gx)
            gy = np.sum(region * Gy)
            edges[i, j] = np.sqrt(gx**2 + gy**2)
    return edges

edges = prewitt_edge(img)
edges = (edges > np.mean(edges)).astype(np.uint8) * 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Prewitt Edge Detector')
plt.show()

# Sobel Edge Detector

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Sobel Edge Detector
def sobel_edge(img):
    height, width = img.shape
    edges = np.zeros_like(img)
    Gx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Gy = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            gx = np.sum(region * Gx)
            gy = np.sum(region * Gy)
            edges[i, j] = np.sqrt(gx**2 + gy**2)
    return edges

edges = sobel_edge(img)
edges = (edges > np.mean(edges)).astype(np.uint8) * 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Sobel Edge Detector')
plt.show()

# Scharr Operator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Scharr Operator
def scharr_edge(img):
    height, width = img.shape
    edges = np.zeros_like(img)
    Gx = np.array([[-3, 0, 3], [-10, 0, 10], [-3, 0, 3]])
    Gy = np.array([[-3, -10, -3], [0, 0, 0], [3, 10, 3]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            gx = np.sum(region * Gx)
            gy = np.sum(region * Gy)
            edges[i, j] = np.sqrt(gx**2 + gy**2)
    return edges

edges = scharr_edge(img)
edges = (edges > np.mean(edges)).astype(np.uint8) * 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Scharr Operator')
plt.show()

# Laplacian Operator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Laplacian Operator
def laplacian_edge(img):
    height, width = img.shape
    edges = np.zeros_like(img)
    kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            edges[i, j] = np.sum(region * kernel)
    return edges

edges = laplacian_edge(img)
edges = np.abs(edges)
edges = (edges > np.mean(edges)).astype(np.uint8) * 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Laplacian Operator')
plt.show()

# Laplacian of Gaussian (LoG)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Laplacian of Gaussian (LoG) - simplified, just Laplacian for now
def laplacian_edge(img):
    height, width = img.shape
    edges = np.zeros_like(img)
    kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            edges[i, j] = np.sum(region * kernel)
    return edges

edges = laplacian_edge(img)
edges = np.abs(edges)
edges = (edges > np.mean(edges)).astype(np.uint8) * 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Laplacian of Gaussian (LoG)')
plt.show()

# Zero-Crossing Method

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Zero-Crossing Method
def laplacian(img):
    height, width = img.shape
    lap = np.zeros_like(img)
    kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            lap[i, j] = np.sum(region * kernel)
    return lap

lap = laplacian(img)
edges = np.zeros_like(img, dtype=np.uint8)
for i in range(1, lap.shape[0]-1):
    for j in range(1, lap.shape[1]-1):
        if (lap[i, j] > 0 and (lap[i-1, j] < 0 or lap[i+1, j] < 0 or lap[i, j-1] < 0 or lap[i, j+1] < 0)) or \
           (lap[i, j] < 0 and (lap[i-1, j] > 0 or lap[i+1, j] > 0 or lap[i, j-1] > 0 or lap[i, j+1] > 0)):
            edges[i, j] = 255

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Zero-Crossing Method')
plt.show()

# Canny Edge Detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# Canny Edge Detection - simplified version
def sobel_edge(img):
    height, width = img.shape
    edges = np.zeros_like(img)
    Gx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Gy = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    for i in range(1, height-1):
        for j in range(1, width-1):
            region = img[i-1:i+2, j-1:j+2]
            gx = np.sum(region * Gx)
            gy = np.sum(region * Gy)
            edges[i, j] = np.sqrt(gx**2 + gy**2)
    return edges

edges = sobel_edge(img)
# Double threshold
high_thresh = np.mean(edges) + np.std(edges)
low_thresh = np.mean(edges) - np.std(edges)
strong = (edges > high_thresh).astype(np.uint8) * 255
weak = ((edges >= low_thresh) & (edges <= high_thresh)).astype(np.uint8) * 127
# Simple hysteresis: keep strong, discard weak
edges = strong

# Display output
plt.imshow(edges, cmap='gray')
plt.title('Canny Edge Detection (simplified)')
plt.show()

# 3. Region-Based Segmentation - Region Growing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Region Growing
def region_growing(img, seed, threshold=10):
    height, width = img.shape
    segmented = np.zeros_like(img)
    visited = np.zeros_like(img, dtype=bool)
    queue = deque([seed])
    visited[seed] = True
    segmented[seed] = 255
    seed_value = img[seed]
    while queue:
        x, y = queue.popleft()
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < height and 0 <= ny < width and not visited[nx, ny]:
                if abs(int(img[nx, ny]) - int(seed_value)) < threshold:
                    visited[nx, ny] = True
                    segmented[nx, ny] = 255
                    queue.append((nx, ny))
    return segmented

seed = (img.shape[0]//2, img.shape[1]//2)
segmented = region_growing(img, seed)

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Region Growing')
plt.show()

# Region Splitting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Region Splitting - simplified
def region_splitting(img, threshold=50):
    height, width = img.shape
    segmented = np.zeros_like(img)
    def split_region(x1, y1, x2, y2):
        region = img[x1:x2, y1:y2]
        if np.var(region) > threshold and (x2 - x1 > 1) and (y2 - y1 > 1):
            mx = (x1 + x2) // 2
            my = (y1 + y2) // 2
            split_region(x1, y1, mx, my)
            split_region(mx, y1, x2, my)
            split_region(x1, my, mx, y2)
            split_region(mx, my, x2, y2)
        else:
            segmented[x1:x2, y1:y2] = np.mean(region)
    split_region(0, 0, height, width)
    return segmented

segmented = region_splitting(img)
segmented = (segmented > np.mean(segmented)).astype(np.uint8) * 255

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Region Splitting')
plt.show()

# Seeded Region Growing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Seeded Region Growing
def seeded_region_growing(img, seeds, threshold=10):
    height, width = img.shape
    segmented = np.zeros_like(img)
    visited = np.zeros_like(img, dtype=bool)
    queue = deque(seeds)
    for seed in seeds:
        visited[seed] = True
        segmented[seed] = 255
    while queue:
        x, y = queue.popleft()
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < height and 0 <= ny < width and not visited[nx, ny]:
                # Check similarity to any seed
                similar = False
                for sx, sy in seeds:
                    if abs(int(img[nx, ny]) - int(img[sx, sy])) < threshold:
                        similar = True
                        break
                if similar:
                    visited[nx, ny] = True
                    segmented[nx, ny] = 255
                    queue.append((nx, ny))
    return segmented

seeds = [(0,0), (0, img.shape[1]-1), (img.shape[0]-1, 0), (img.shape[0]-1, img.shape[1]-1)]
segmented = seeded_region_growing(img, seeds)

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Seeded Region Growing')
plt.show()

# 4. Clustering-Based Segmentation - K-Means Clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.float32)

# K-Means Clustering
def kmeans_segmentation(img, k=3, max_iter=10):
    height, width = img.shape
    pixels = img.ravel()
    # Random centroids
    centroids = np.random.choice(pixels, k, replace=False)
    for _ in range(max_iter):
        labels = np.zeros(len(pixels))
        for i, pixel in enumerate(pixels):
            distances = np.abs(pixel - centroids)
            labels[i] = np.argmin(distances)
        for j in range(k):
            cluster_pixels = pixels[labels == j]
            if len(cluster_pixels) > 0:
                centroids[j] = np.mean(cluster_pixels)
    segmented = centroids[labels.astype(int)].reshape(height, width)
    return segmented

segmented = kmeans_segmentation(img)
segmented = (segmented / 255 * 255).astype(np.uint8)

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('K-Means Clustering')
plt.show()

# 6. Watershed Segmentation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Read example image
img = plt.imread('images/butterfly.jpg')

# Convert to grayscale if necessary
if img.ndim == 3:
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
img = img.astype(np.uint8)

# Watershed Segmentation - simplified using multiple region growing
def watershed_segmentation(img, num_seeds=10):
    height, width = img.shape
    segmented = np.zeros_like(img)
    seeds = []
    for _ in range(num_seeds):
        x = np.random.randint(0, height)
        y = np.random.randint(0, width)
        seeds.append((x, y))
    # Assign each seed a label
    labels = np.zeros_like(img, dtype=int)
    for i, seed in enumerate(seeds):
        labels[seed] = i + 1
    # Grow regions
    queue = deque(seeds)
    visited = np.zeros_like(img, dtype=bool)
    for seed in seeds:
        visited[seed] = True
    while queue:
        x, y = queue.popleft()
        current_label = labels[x, y]
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < height and 0 <= ny < width and not visited[nx, ny]:
                if abs(int(img[nx, ny]) - int(img[x, y])) < 20:  # threshold
                    visited[nx, ny] = True
                    labels[nx, ny] = current_label
                    queue.append((nx, ny))
    # Color by label
    segmented = (labels / np.max(labels) * 255).astype(np.uint8)
    return segmented

segmented = watershed_segmentation(img)

# Display output
plt.imshow(segmented, cmap='gray')
plt.title('Watershed Segmentation (simplified)')
plt.show()